# My notes from the course


In [4]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain


In [9]:
%pip install -U langchain-openai -quiet


Usage:   
  /home/elzbieta/genai/LangChain-for-LLM-Application-Development/.venv/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /home/elzbieta/genai/LangChain-for-LLM-Application-Development/.venv/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /home/elzbieta/genai/LangChain-for-LLM-Application-Development/.venv/bin/python -m pip install [options] [-e] <vcs project url> ...
  /home/elzbieta/genai/LangChain-for-LLM-Application-Development/.venv/bin/python -m pip install [options] [-e] <local project path> ...
  /home/elzbieta/genai/LangChain-for-LLM-Application-Development/.venv/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


# Langchain has reusable prompt templates

In [19]:
from langchain.prompts import ChatPromptTemplate
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

In [20]:
customer_messages = prompt_template.format_messages(
                    style="excited French",
                    text="Hello, my name is Andrew")

In [21]:
llm = ChatOpenAI(temperature=0.0)

In [24]:
customer_messages_translated = llm.invoke(customer_messages)

In [25]:
print(customer_messages_translated)

content="Bonjour, je m'appelle Andrew!" response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


In [30]:
customer_messages = prompt_template.format_messages(
                    style="very formal German",
                    text="Hello, my name is Andrew")

In [31]:
res=llm.invoke(customer_messages)

it looks like I can invoke prompts but also directly strings

In [34]:
llm.invoke("What is my name")

AIMessage(content="I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information.", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

# Prompts can have output parsers

See L1 notebook, output parsers, for how to extract directly a JSON from the LLM output.

# Simple ConversationChain
By default, the ConversationChain has a simple type of memory that remembers all previous inputs/outputs and adds them to the context that is passed to the LLM (see ConversationBufferMemory)

In [5]:
llm = ChatOpenAI(temperature=0.0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
)

In [6]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [7]:
conversation.predict(input="What is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is my name
AI:

> Finished chain.


'Your name is Andrew.'

# Does openAI now have memory by default? Somehow with the key?

The answer is :NO.
To prove it, I want to do 2 accesses and check if it remembers the name from the 1.st access

In [8]:
from openai import OpenAI
client = OpenAI()
client.chat.completions.create()

In [12]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [13]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

In [14]:
get_completion("My name is andrew")

'Hello Andrew! How can I assist you today?'

In [15]:
get_completion("What is my name?")

"I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."

So, if I make to API requests, there is no memory between them

In [37]:
# Test sending multiple prompts to openAI

messages = [{"role": "user", "content": "tell me about Warsaw uprising"}, {"role": "user", "content": "Does your tooth hurt"}, ]
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, 
    )
print(response)

ChatCompletion(id='chatcmpl-96h62ziDJuFVwLC2O4IVLWNEU6pig', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm an AI and I don't have teeth, so I can't experience tooth pain. Is there anything else you would like to know or discuss?", role='assistant', function_call=None, tool_calls=None))], created=1711382658, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=31, prompt_tokens=20, total_tokens=51))


My little experiment did not work, only the second message got processed. TODO: understand the usage of messages being a list

# ConverstationChain with explicit memory


In [ ]:
conversation2 = ConversationChain(
    llm=llm, 
    verbose=True,
    memory = memory
)